<a href="https://colab.research.google.com/github/Miit-009/Jio-Institute/blob/main/Session_9_(11_Feb)_Thumbnails_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install face-recognition dlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=45d6b8a37df0a9d69b83cf3b77d81a8fec4e7b63fea66bfd1afcd1a0c16914b0
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [ ]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from skimage import exposure
from skimage.color import rgb2hsv
from skimage.filters import gaussian
from skimage.exposure import is_low_contrast
from skimage.measure import shannon_entropy
from PIL import Image

# Load OpenCV's Haar cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def analyze_image(image_path):
    image_path = Path(image_path)  # Convert string path to Path object
    image = cv2.imread(str(image_path))
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces using OpenCV
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Face details
    face_present = len(faces) > 0
    num_faces = len(faces)
    face_size_ratios = [(w * h) / (image.shape[0] * image.shape[1]) for (x, y, w, h) in faces]

    # Brightness
    brightness = np.mean(gray_image)

    # Contrast
    contrast = gray_image.std()

    # Hue, Saturation (HSV conversion)
    hsv_image = rgb2hsv(rgb_image)
    hue = np.mean(hsv_image[:, :, 0])
    saturation = np.mean(hsv_image[:, :, 1])

    # Sharpness
    laplacian_var = cv2.Laplacian(gray_image, cv2.CV_64F).var()

    # White Balance Deviation
    mean_r, mean_g, mean_b = np.mean(rgb_image, axis=(0, 1))
    white_balance_deviation = max(abs(mean_r - mean_g), abs(mean_g - mean_b), abs(mean_b - mean_r))

    # Exposure
    exposure_value = np.mean(rgb_image) / 255.0

    # Aspect Ratio
    height, width = image.shape[:2]
    aspect_ratio = width / height
    resolution = width * height

    # Gamma Estimation
    estimated_gamma = exposure.adjust_gamma(image, gamma=1).mean() / 255.0

    # Check for low contrast
    low_contrast = is_low_contrast(gray_image)

    return {
        "video_id": image_path.stem,
        "face_present": face_present,
        "num_faces": num_faces,
        "face_size_ratios": face_size_ratios if face_size_ratios else None,
        "dominant_emotions": "N/A",  # Requires an external emotion detection library
        "width": width,
        "height": height,
        "resolution": resolution,
        "aspect_ratio": aspect_ratio,
        "brightness": brightness,
        "contrast": contrast,
        "saturation": saturation,
        "hue": hue,
        "estimated_gamma": estimated_gamma,
        "sharpness": laplacian_var,
        "white_balance_deviation": white_balance_deviation,
        "exposure": exposure_value
    }

def process_images_colab(output_excel):
    from google.colab import files
    import os

    folder_path = "/content"
    image_files = [f for f in os.listdir(folder_path) if f.endswith(".jpg")]
    data = [analyze_image(os.path.join(folder_path, image)) for image in image_files]
    df = pd.DataFrame(data)
    df.to_excel(output_excel, index=False)
    files.download(output_excel)
    print(f"Excel file saved and downloaded: {output_excel}")

# Usage
process_images_colab("video_analysis.xlsx")
